In [24]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as sm
import statsmodels.api as old_sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from scipy import stats
from sklearn import preprocessing
from sklearn.decomposition import PCA

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Define Heaviside function as H(x)=0 for x<0, H(x)=1/2 for x=0, H(x)=1 for x>0
def heaviside(x):
    return 0.5 * (np.sign(x) + 1)

# Import data
try:
    all_seasons = pd.read_csv("Long_History.csv", encoding='utf-8', index_col=0)
    
    # Write DF to give teams numerical ID, and as lookup for team names and their ID
    teamlist = all_seasons.loc[:, "TeamH"].unique()
    data = np.array([range(len(teamlist)), teamlist]).T
    team_df = pd.DataFrame(data=data, columns=['TID', 'TName'])

except:
    print "Data import failed"

In [25]:
cleaned_df = pd.merge(team_df, all_seasons, how='inner', left_on='TName', right_on='TeamH')
cleaned_df = pd.merge(team_df, cleaned_df, how='inner', left_on='TName', right_on='TeamA', suffixes=('_A', '_H'))

columns_cl = ['Season', 'Gameday', 'TID_H', 'TID_A', 'FTHG', 'FTAG']
cleaned_df = cleaned_df.loc[:, columns_cl].sort_values(['Season', 'Gameday'])
cleaned_df.index = range(len(cleaned_df))

cleaned_df.loc[:, 'FTGD'] = cleaned_df.loc[:, 'FTHG'] - cleaned_df.loc[:, 'FTAG']

cleaned_df = cleaned_df.applymap(int)

In [39]:
def standings(df):
    def points(x):
        if x > 0:
            return 3
        elif x == 0:
            return 1
        else:
            return 0
    multi_index = pd.MultiIndex(levels=[[],[],[]],
                                labels=[[],[],[]],
                                names=[u'TID', u'Season', u'Gameday'])
    table_df = pd.DataFrame(data=None, columns=['Points', 'GD'], index=multi_index)
    
    for tid in range(len(team_df)):
        query = 'TID_H == ' + str(tid)
        temp_H = df.query(query).loc[:, ['TID_H', 'Season', 'Gameday', 'FTGD']]
        temp_H.loc[:, 'GD'] = temp_H.loc[:, 'FTGD']
        temp_H = temp_H.drop('FTGD', axis=1)
        temp_H = temp_H.set_index(['TID_H', 'Season', 'Gameday'])

        query = 'TID_A == ' + str(tid)
        temp_A = df.query(query).loc[:, ['TID_A', 'Season', 'Gameday', 'FTGD']]
        temp_A.loc[:, 'GD'] = -1 * temp_A.loc[:, 'FTGD']
        temp_A = temp_A.drop('FTGD', axis=1)
        temp_A = temp_A.set_index(['TID_A', 'Season', 'Gameday'])

        temp_df = pd.concat([temp_H, temp_A]).sort_index()
        temp_df.index = temp_df.index.rename(['TID', 'Season', 'Gameday'])
        temp_df.loc[:, 'Points'] = temp_df.loc[:, 'GD'].apply(points).rename('Points').to_frame()

        table_df = table_df.append(temp_df.groupby(level=['TID', 'Season'], group_keys=False).cumsum())
    
    return table_df

table_df = standings(cleaned_df)

# Season end standings
# Update most recent season such that we calculate it as if 34 Games have been played.
table_extrap_df = \
    table_df.groupby(level=['Season', 'TID']).last()/table_df.groupby(level=['Season', 'TID']).count()

In [40]:
table_extrap_df

GD    Points
Season TID                    
1965   0   -2.735294  0.294118
       1   -0.176471  1.323529
       2   -1.470588  0.911765
       3    1.176471  2.058824
       4    0.970588  1.852941
       5    0.529412  1.588235
       6    0.647059  1.470588
       7    1.058824  1.941176
       8    0.000000  1.323529
       9   -0.647059  1.088235
       10   0.323529  1.558824
       11   0.970588  1.970588
       12  -0.323529  1.117647
       13  -0.676471  1.000000
       14   0.058824  1.205882
       15   0.352941  1.382353
       16   1.000000  1.941176
       17  -1.058824  0.970588
1966   1   -0.176471  1.264706
       3    0.382353  1.705882
       4    0.000000  1.500000
       5    0.500000  1.588235
       6   -0.058824  1.264706
       7   -0.205882  1.147059
       8    0.647059  1.764706
       9   -0.764706  1.235294
       10  -0.205882  1.352941
       11   0.441176  1.558824
       12   0.617647  1.352941
       13   0.029412  1.500000
...              ...       ...
2015   12   0.500000  1.617647
       14  -0.911765  0.735294
       15  -0.176471  1.205882
       16   1.411765  2.294118
       22   0.000000  1.470588
       32  -0.441176  1.117647
       33   0.470588  1.764706
       43  -0.058824  1.323529
       47   0.117647  1.470588
       48  -0.441176  1.088235
       49  -0.294118  1.117647
       52  -0.264706  1.176471
2016   4    0.523810  1.571429
       5    0.238095  1.666667
       7   -0.761905  0.904762
       9    0.142857  1.238095
       11   1.571429  2.380952
       12  -0.285714  1.238095
       15  -0.761905  0.952381
       16   0.952381  1.761905
       22   0.190476  1.619048
       32  -1.285714  0.571429
       33   0.190476  1.428571
       41  -0.380952  1.428571
       43  -0.571429  1.047619
       47  -0.380952  1.190476
       48   0.809524  1.761905
       49  -0.380952  1.142857
       52  -0.619048  0.857143
       53   0.809524  2.142857

[918 rows x 2 columns]